In [2]:
import tensorflow as tf

In [3]:
import numpy as np
import matplotlib.pyplot as plt
import plotly.express as px
import seaborn as sns
from sklearn.metrics import classification_report, accuracy_score

In [4]:
train_dir = '../input/blood-cells/dataset2-master/dataset2-master/images/TRAIN'
test_dir = '../input/blood-cells/dataset2-master/dataset2-master/images/TEST'

In [5]:
#Create Generators

train_gen = tf.keras.preprocessing.image.ImageDataGenerator(
    preprocessing_function = tf.keras.applications.mobilenet_v2.preprocess_input,
    validation_split= 0.2)


test_gen = tf.keras.preprocessing.image.ImageDataGenerator(
    preprocessing_function = tf.keras.applications.mobilenet_v2.preprocess_input)

In [6]:
#Flow image data
train_images = train_gen.flow_from_directory(
    directory=train_dir,
    target_size = (224,224),
    color_mode='rgb',
    class_mode='categorical',
    batch_size=32,
    seed=42,
    shuffle=True,
    subset='training')

val_images = train_gen.flow_from_directory(
    directory=train_dir,
    target_size = (224,224),
    color_mode='rgb',
    class_mode='categorical',
    batch_size=32,
    seed=42,
    shuffle=True,
    subset='validation')

test_images = test_gen.flow_from_directory(
    directory=test_dir,
    target_size = (224,224),
    color_mode='rgb',
    class_mode='categorical',
    batch_size=32,
    seed=42,
    shuffle=False)

Found 7968 images belonging to 4 classes.
Found 1989 images belonging to 4 classes.
Found 2487 images belonging to 4 classes.


In [7]:
pretrained_model = tf.keras.applications.MobileNetV2(
    input_shape=(224,224,3),
    include_top=False,
    weights='imagenet',
    pooling='avg')

pretrained_model.trainable = False

9412608/9406464 [==============================] - 0s 0us/step


In [8]:
inputs = pretrained_model.input
x = tf.keras.layers.Dense(128, activation='relu')(pretrained_model.output)
outputs = tf.keras.layers.Dense(4, activation='softmax')(x)

model = tf.keras.Model(inputs = inputs, outputs= outputs)

model.compile(optimizer = 'adam',
             loss='categorical_crossentropy',
             metrics = ['accuracy'])

print(model.summary())

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 224, 224, 3) 0                                            
__________________________________________________________________________________________________
Conv1 (Conv2D)                  (None, 112, 112, 32) 864         input_1[0][0]                    
__________________________________________________________________________________________________
bn_Conv1 (BatchNormalization)   (None, 112, 112, 32) 128         Conv1[0][0]                      
__________________________________________________________________________________________________
Conv1_relu (ReLU)               (None, 112, 112, 32) 0           bn_Conv1[0][0]                   
______________________________________________________________________________________________

In [9]:
#Training
history = model.fit(
    train_images,
    validation_data= val_images,
    epochs = 100,
    callbacks = [
        tf.keras.callbacks.EarlyStopping(
        monitor = 'val_loss',
        patience=3,
        restore_best_weights=True)
    ])

Epoch 1/100
249/249 [==============================] - 256s 1s/step - loss: 1.0077 - accuracy: 0.5779 - val_loss: 0.6105 - val_accuracy: 0.7627
Epoch 2/100
249/249 [==============================] - 253s 1s/step - loss: 0.4840 - accuracy: 0.8151 - val_loss: 0.4353 - val_accuracy: 0.8311
Epoch 3/100
249/249 [==============================] - 250s 1s/step - loss: 0.3550 - accuracy: 0.8692 - val_loss: 0.4615 - val_accuracy: 0.8165
Epoch 4/100
249/249 [==============================] - 271s 1s/step - loss: 0.2916 - accuracy: 0.8910 - val_loss: 0.3431 - val_accuracy: 0.8643
Epoch 5/100
249/249 [==============================] - 254s 1s/step - loss: 0.2331 - accuracy: 0.9147 - val_loss: 0.2972 - val_accuracy: 0.8849
Epoch 6/100
249/249 [==============================] - 252s 1s/step - loss: 0.1836 - accuracy: 0.9352 - val_loss: 0.2738 - val_accuracy: 0.8964
Epoch 7/100
249/249 [==============================] - 275s 1s/step - loss: 0.1474 - accuracy: 0.9518 - val_loss: 0.2385 - val_accuracy:

In [11]:
fig = px.line(
    history.history,
    y=['loss', 'val_loss'],
    labels={'index': "Epoch", 'value': "Loss"},
    title="Training and Validation Loss Over Time"
)

fig.show()

In [12]:
CLASS_NAMES = list(train_images.class_indices.keys())
CLASS_NAMES

['EOSINOPHIL', 'LYMPHOCYTE', 'MONOCYTE', 'NEUTROPHIL']

In [ ]:
predictions = np.argmax(model.predict(test_images), axis=1)

acc = accuracy_score(test_images.labels, predictions)
cm = tf.math.confusion_matrix(test_images.labels, predictions)
clr = classification_report(test_images.labels, predictions, target_names=CLASS_NAMES)

print("Test Accuracy: {:.3f}%".format(acc * 100))

plt.figure(figsize=(8, 8))
sns.heatmap(cm, annot=True, fmt='g', vmin=0, cmap='Blues', cbar=False)
plt.xticks(ticks= np.arange(4) + 0.5, labels=CLASS_NAMES)
plt.yticks(ticks= np.arange(4) + 0.5, labels=CLASS_NAMES)
plt.xlabel("Predicted")
plt.ylabel("Actual")
plt.title("Confusion Matrix")
plt.show()

print("Classification Report:\n----------------------\n", clr)